In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from distortions.geometry import neighborhoods
from distortions.visualization import dplot
import numpy as np
import scanpy as sc

In [ ]:
# Read data directly from GitHub
data_url = "https://raw.githubusercontent.com/krisrs1128/distortions-data/main/data/c-elegans_qc_final.txt"
metadata_url = "https://raw.githubusercontent.com/krisrs1128/distortions-data/main/data/c-elegans_qc_final_metadata.txt"

data = np.loadtxt(data_url, delimiter="\t")
metadata = pd.read_csv(metadata_url, sep=",")

print("Data shape:", data.shape)
print("Metadata shape:", metadata.shape)

In [ ]:
picked_cell_type = ["Ciliated_amphid_neuron", "Pharyngeal_neuron", "GLR", "Glia",
                    "Body_wall_muscle", "Pharyngeal_muscle","Hypodermis", "Intestine"]

'''
Biological meaning for above choise.
Ciliated_Amphid_Neuron  ->  Glia
        ->
     Pharyngeal_Neuron  <->  Pharyngeal_Muscle  ->  Intestine
        ->
        ->  GLR  <->  Body_Wall_Muscle
            ->
            Hypodermis (outer interface)
The Arrow above means the functional or developmental interactions between cell types
'''

In [ ]:
cat = pd.Categorical(metadata['cell.type'])
colors = cat.codes
label_pick = [i for i, name in enumerate(cat.categories) if name in picked_cell_type]

from collections import defaultdict
def sampling(source, num_sample, label_pick):
    class_indices = defaultdict(list)
    for idx, label in enumerate(source):
        class_indices[label].append(idx)

    rng = np.random.default_rng(seed=42)  # Set a seed for reproducibility
    sampled_indices = []
    for label, indices in class_indices.items():
        if label in label_pick:
            n_class_samples = min(len(indices), num_sample)  # Equal sampling
            sampled_indices.extend(list(rng.choice(indices, n_class_samples, replace=False)))
    return sampled_indices

sampled_indices = sampling(colors, 100, label_pick)

In [ ]:
from scipy.spatial import KDTree
import os

cache_file = "data/c_elegans_distances.npy"
if os.path.exists(cache_file):
    dists = np.load(cache_file)
else:
    tree = KDTree(data)
    dists, _ = tree.query(data, k=3)
    os.makedirs("data", exist_ok=True)
    np.save(cache_file, dists)

In [ ]:
np.random.seed(42)
import umap
embedding_dumap = umap.UMAP(n_neighbors=10, n_components=2, n_epochs=500, densmap = True).fit_transform(data)
embedding_umap = umap.UMAP(n_neighbors=10, n_components=2, n_epochs=500, densmap = False).fit_transform(data)

In [ ]:
plt.figure(figsize=(8, 6))
fig_umpa =plt.scatter(embedding_umap[:, 0], embedding_umap[:, 1],
            c=pd.Categorical(metadata['cell.type']).codes, cmap='viridis', s=1)
plt.colorbar(fig_umpa, label='Cell Type')
plt.title("UMAP Embedding of C.elegans Dataset")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
fig_umpa =plt.scatter(embedding_dumap[:, 0], embedding_dumap[:, 1],
            c=pd.Categorical(metadata['cell.type']).codes, cmap='viridis', s=1)
plt.colorbar(fig_umpa, label='Cell Type')
plt.title("DenseUMAP Embedding of C.elegans Dataset")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()


In [ ]:
import anndata as ad
adata = ad.AnnData(X=data[sampled_indices])
adata.X.shape
adata.obsm["X_UMAP"] = embedding_umap[sampled_indices]
adata.obsm["X_DenseUMAP"] = embedding_dumap[sampled_indices]
adata.obs["cell_type"] = metadata['cell.type'].values[sampled_indices]
sc.pp.neighbors(adata, n_neighbors=50, use_rep='X', method='gauss')

In [ ]:
from distortions.geometry import Geometry, bind_metric, local_distortions

radius = 3 * np.mean(dists)
umap_embed_test = adata.obsm["X_UMAP"].copy()
geom = Geometry("brute", laplacian_method="geometric", affinity_kwds={"radius": radius}, adjacency_kwds={"radius": None, "n_neighbors": 10}, laplacian_kwds={"scaling_epps": radius})
H, Hvv, Hs = local_distortions(umap_embed_test, adata.X, geom)
umap_embed_test = bind_metric(umap_embed_test, Hvv, Hs)
umap_embed_test["cell_type"] = adata.obs["cell_type"].values
summary = {"umap_kappa": Hs[:, 0] / Hs[:, 1], "umap_vol": Hs[:, 0] * Hs[:, 1]}


umap_N = neighborhoods(adata, threshold=.2, outlier_factor=2, method="box", embed_key="X_UMAP")
UMAP_p = dplot(umap_embed_test, width=600, height=500)\
    .mapping(x="embedding_0", y="embedding_1", color="cell_type")\
    .geom_ellipse(radiusMax=8, radiusMin=2)\
    .scale_color(legendTextSize=8)\
    .labs(x="UMAP 1", y="UMAP 2")\
    .inter_edge_link(N=umap_N, threshold=1, backgroundOpacity=0.4, strokeWidth=0.1, strokeOpacity=1, highlightStrokeWidth=0.1)
UMAP_p

In [ ]:
dumap_embed_test = adata.obsm["X_DenseUMAP"].copy()
geom = Geometry("brute", laplacian_method="geometric", affinity_kwds={"radius": radius}, adjacency_kwds={"radius": None, "n_neighbors": 10}, laplacian_kwds={"scaling_epps": radius})
H, Hvv, Hs = local_distortions(dumap_embed_test, adata.X, geom)
dumap_embed_test = bind_metric(dumap_embed_test, Hvv, Hs)
dumap_embed_test["cell_type"] = adata.obs["cell_type"].values
summary["densumap_kappa"] = Hs[:, 0] / Hs[:, 1]
summary["densumap_vol"] = Hs[:, 0] * Hs[:, 1]


Densumap_N = neighborhoods(adata, threshold=.2, outlier_factor=2, method="box", embed_key="X_DenseUMAP")
DensMAP_p = dplot(dumap_embed_test, width=600, height=500)\
    .mapping(x="embedding_0", y="embedding_1", color="cell_type")\
    .geom_ellipse(radiusMax=8, radiusMin=2)\
    .scale_color(legendTextSize=8)\
    .labs(x="UMAP 1", y="UMAP 2")\
    .inter_edge_link(N=Densumap_N, threshold=1, backgroundOpacity=0.4, strokeWidth=0.1, strokeOpacity=1, highlightStrokeWidth=0.1)
DensMAP_p

In [ ]:
umap_embed_test

In [ ]:
#metrics = {k: H[k] for k in range(len(H))}
umap_hair = dplot(umap_embed_test, width=700, height=500)\
    .mapping(x="embedding_0", y="embedding_1", angle="angle", a="s0", b="s1", color="cell_type")\
    .geom_hair()
umap_hair

In [ ]:
metrics = {k: H[k] for k in range(len(H))}
dense_hair = dplot(dumap_embed_test, width=700, height=500)\
    .mapping(x="embedding_0", y="embedding_1", angle="angle", a="s0", b="s1", color="cell_type")\
    .geom_hair()\
    .scale_color(legendTextSize=8)\
    .labs(x="x-axis", y="y-axis")
dense_hair

In [ ]:
import pandas as pd
import altair as alt

summary["densmap_kappa"] = Hs[:, 0] / Hs[:, 1]
summary["densmap_vol"] = Hs[:, 0] * Hs[:, 1]
df_kappa = pd.concat([
    pd.DataFrame({'ratio': np.log(summary["umap_kappa"]), 'method': 'UMAP', 'cell_type': adata.obs['cell_type']}),
    pd.DataFrame({'ratio': np.log(summary["densmap_kappa"]), 'method': 'DensMAP'})
])

p = alt.Chart(df_kappa).mark_bar(opacity=0.5).encode(
    x=alt.X('ratio', bin=alt.Bin(maxbins=100), title='Log of Condition Number'),
    y=alt.Y('count()', stack=None, title='Count of Cells'),
    color=alt.Color('method:N', legend=alt.Legend(title="Method"))
).properties(width=400, height=300)

save_dir = "/Users/krissankaran/Desktop/collaborations/distortions-project/distortions-dev/paper/figures/"
p.save(f"{save_dir}/condition_number_ratios.svg")

In [ ]:
umap_distortion_numbers = []
densmap_distortion_numbers = []
for i in np.linspace(0.05, 0.95, num=19):
    umap_distortion_numbers.append(len(neighborhoods(adata, threshold=i, outlier_factor=2, method="box", embed_key="X_UMAP")))
    densmap_distortion_numbers.append(len(neighborhoods(adata, threshold=i, outlier_factor=2, method="box", embed_key="X_DenseUMAP")))

In [ ]:
df_distortion = pd.concat([
    pd.DataFrame({'threshold': np.linspace(0.05, 0.95, num=19), 'num_distortions': umap_distortion_numbers, 'method': 'umap'}),
    pd.DataFrame({'threshold': np.linspace(0.05, 0.95, num=19), 'num_distortions': densmap_distortion_numbers, 'method': 'densmap'})
])
p = alt.Chart(df_distortion).mark_bar().encode(
    x=alt.X('threshold:Q', bin=alt.Bin(maxbins=20), title='Threshold Fraction'),
    y=alt.Y('num_distortions:Q', title='Number of Fragmented Neighborhoods'),
    color=alt.Color('method:N', legend=None)
).properties(width=400, height=300)

p.save(f"{save_dir}/threshold_fractions.svg")